Required Packages for extracting data from youtube videos

In [1]:
%pip install yt-dlp


Note: you may need to restart the kernel to use updated packages.


Collecting Data For Correct and Incorrect Squats pose

In [2]:
import os

video_url = "https://youtu.be/gzd9gcsr4Bs?si=JJqa45uShBd-LuUq"  # Example: squat
video_filename = "squat_10.mp4"

# Download best available mp4 video
os.system(f'yt-dlp -f "best[ext=mp4]" -o "{video_filename}" {video_url}')
print("Downloaded:", video_filename)


Downloaded: squat_10.mp4


In [7]:
import cv2
import mediapipe as mp
import pandas as pd

# Setup MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_draw = mp.solutions.drawing_utils

# Load video
cap = cv2.VideoCapture("squat_10.mp4")
data = []

print("Controls:")
print("  Press 'c' → Correct posture")
print("  Press 'i' → Incorrect posture")
print("  Press 'n' → Skip frame")
print("  Press 'q' → Quit and save")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB for MediaPipe
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(img_rgb)

    # Draw pose landmarks
    if result.pose_landmarks:
        mp_draw.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Show frame
    cv2.imshow("Pose Labeling - squat_4.mp4", cv2.resize(frame, (960, 540)))

    # Wait for key
    key = cv2.waitKey(0) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('n'):
        continue
    elif key == ord('c'):
        label = 1
    elif key == ord('i'):
        label = 0
    else:
        continue

    # Extract pose landmarks
    if result.pose_landmarks:
        row = []
        for lm in result.pose_landmarks.landmark:
            row.extend([lm.x, lm.y, lm.visibility])
        row.append(label)
        data.append(row)
        print(f"Labeled frame as: {'Correct' if label == 1 else 'Incorrect'}")

# Release resources
cap.release()
cv2.destroyAllWindows()

# Save to CSV
cols = [f'{i}_{axis}' for i in range(33) for axis in ['x', 'y', 'v']] + ['label']
df = pd.DataFrame(data, columns=cols)
df.to_csv("squat_6.csv", index=False)

print("✅ Saved dataset to: squat_2.csv")


Controls:
  Press 'c' → Correct posture
  Press 'i' → Incorrect posture
  Press 'n' → Skip frame
  Press 'q' → Quit and save
Labeled frame as: Correct
Labeled frame as: Correct
Labeled frame as: Correct
Labeled frame as: Correct
Labeled frame as: Correct
Labeled frame as: Correct
Labeled frame as: Correct
Labeled frame as: Correct
✅ Saved dataset to: squat_2.csv


In [8]:
import pandas as pd
import glob

# Step 1: List of all CSV files to merge
csv_files = [
    "squat_1.csv",
    "squat_2.csv",
    "squat_3.csv",
    "squat_4.csv",
    "squat_5.csv",
    "squat_6.csv"
]

# Step 2: Load and concatenate all CSVs
dataframes = [pd.read_csv(file) for file in csv_files]
merged_df = pd.concat(dataframes, ignore_index=True)

# Step 3: Save to final merged CSV
merged_df.to_csv("squat_final.csv", index=False)

print("✅ All files merged into squat_final.csv")


✅ All files merged into squat_final.csv
